In [1]:
import os
from getpass import getpass

import openai

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
import pandas as pd

# Display the complete contents of dataframe cells.
pd.set_option("display.max_colwidth", None)

In [3]:
import requests

In [4]:
import phoenix as px
from llama_index.core import set_global_handler
from phoenix.trace.langchain import LangChainInstrumentor


session = px.launch_app()

# Setup instrumentation for both llama-index and LangChain (used by Ragas)
set_global_handler("arize_phoenix")
LangChainInstrumentor().instrument()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
# !git clone https://huggingface.co/datasets/explodinggradients/prompt-engineering-papers

In [6]:
# !mv prompt-engineering-papers/ data/prompt-engineering-papers

In [7]:

from llama_index.core import SimpleDirectoryReader

dir_path = "./data/prompt-engineering-papers"
reader = SimpleDirectoryReader(dir_path, num_files_limit=2)
documents = reader.load_data()

In [8]:
from llama_index.llms.ollama import Ollama
model = "open-hermes-2-4_0"

In [9]:
llm = Ollama(model=model, request_timeout=30.0)

In [10]:
resp = llm.complete("Who is Cedric Bixler-Zavala?")

In [11]:
print(resp)

Cedric Bixler-Zavala is a musician best known for being the vocalist for the bands At The Drive-In, The Mars Volta, and Zavalaz. He has received critical acclaim for his unique vocal style and has been recognized as one of the most innovative and influential musicians in modern rock music.


In [12]:
from phoenix.trace import using_project
from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import TestsetGenerator
from ragas.llms.base import LangchainLLMWrapper, OpenAIEmbeddings


In [13]:
wrapped_ollama_model = LangchainLLMWrapper(llm)

In [14]:
wrapped_ollama_model

LangchainLLMWrapper(run_config=RunConfig(timeout=60, max_retries=10, max_wait=60, max_workers=16, exception_types=<class 'Exception'>))

In [29]:
TEST_SIZE = 1

generator_llm = wrapped_ollama_model
critic_llm = wrapped_ollama_model
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)


In [30]:

# set question type distribution
distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}


In [31]:

# generate testset
with using_project("ragas-testset"):
    testset = generator.generate_with_llamaindex_docs(
        documents, test_size=TEST_SIZE, distributions=distribution
    )
test_df = testset.to_pandas()
test_df.head()

embedding nodes:   0%|          | 0/144 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/ragas/executor.py", line 96, in run
    results = self.loop.run_until_complete(self._aresults())
  File "uvloop/loop.pyx", line 1517, in uvloop.loop.Loop.run_until_complete
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/ragas/executor.py", line 84, in _aresults
    raise e
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
  File "/usr/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/ragas/executor.py", line 38, in se

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.